In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 10 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Loi de conservation scalaire : cas linéaire
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "seaborn"

# Loi de conservation scalaire : cas linéaire

## Equation d'advection linéaire

$$ \partial_t u(x,t) + a \, \partial_x u(x,t) = 0$$

Nous prendrons comme vitesse de propagation $a=1$ et une donnée initiale type "petit chapeau"

In [ ]:
# Solution initiale du petit chapeau u0
def hat_ini(x):
    u = np.empty(x.size)
    for ix, xi in enumerate(x):
        if xi <= -1 or xi>=1:
            u[ix] = 0
        elif xi < 0:
            u[ix] = xi + 1
        else:
            u[ix] = -xi + 1
    return u

# Solution initiale du créneau
def step_ini(x):
    u = np.empty(x.size)
    for ix, xi in enumerate(x):
        if xi <= 0 or xi>=2:
            u[ix] = -0.5
        else:
            u[ix] = 1.
    return u

In [ ]:
def upwind(u, nt, dtoverdx):
    for it in range(nt-1):
        flux = a*u
        u[1:] += -dtoverdx * (flux[1:] - flux[:-1])
        u[0] = u[-1]
    return u

### Influence du CFL

In [ ]:
xmin = -2.0
xmax =  6.0

tini = 0.0
tend = 4.0

a = 1.0

uini = hat_ini

fig = go.Figure()
nxplot = 1000
dxplot = (xmax-xmin) / nxplot
xplot = np.linspace(xmin+(dxplot/2), xmax-(dxplot/2), 2000)
fig.add_trace(go.Scatter(x=xplot, y=uini(xplot), name='initial sol.'))
fig.add_trace(go.Scatter(x=xplot, y=uini(a*xplot-tend), name='exact sol.'))

nx = 250
dx = (xmax-xmin) / nx
x = np.linspace(xmin+(dx/2), xmax-(dx/2), nx)

u = np.zeros(nx+1)

cfl = [0.1, 0.25, 0.5, 0.9, 1.0, 1.01]
for cfl_i in cfl:
    
    nt = int(a*(tend-tini)/(cfl_i*dx)) + 1
    dt = (tend-tini)/(nt-1)
    dtoverdx = dt/dx
    
    u[1:] = uini(x)
    u = upwind(u, nt, dtoverdx)
    
    marker=dict(symbol='x')
    fig.add_trace(go.Scatter(x=x, y=u[1:], name='num sol.', visible=False, mode='markers', marker=marker))

fig.data[2].visible = True

# Create and add slider
steps = []
for i, cfl_i in enumerate(cfl):
    nt = int(a*(tend-tini)/(cfl_i*dx)) + 1
    dt = (tend-tini)/(nt-1)
    args = [{"visible": [(el==0 or el==1 or el==i+2) for el in range(len(fig.data))]}]
    step = dict(method="update", label = f"{cfl_i}", args=args)
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'cfl = '}, steps=steps)]

fig.update_layout(sliders=sliders, title=f"Schéma upwind : nx={nx}")
fig.show()

### Solution à CFL fixée

In [ ]:
xmin = -2.0
xmax =  6.0

tini = 0.0
tend = 4.0

cfl = 0.9

a = 1.0

uini = hat_ini

fig = go.Figure()
nxplot = 2000
dxplot = (xmax-xmin) / nxplot
xplot = np.linspace(xmin+(dxplot/2), xmax-(dxplot/2), nxplot)
fig.add_trace(go.Scatter(x=xplot, y=uini(xplot), name='initial sol.'))
fig.add_trace(go.Scatter(x=xplot, y=uini(a*xplot-tend), name='exact sol.'))

nx = [125, 250, 500, 1000, 2000]
for nx_i in nx:
    dx = (xmax-xmin) / nx_i
    x = np.linspace(xmin+(dx/2), xmax-(dx/2), nx_i)
    
    nt = int(a*(tend-tini)/(cfl*dx)) + 1
    dt = (tend-tini)/(nt-1)
    dtoverdx = dt/dx
    
    u = np.zeros(nx_i+1)
    u[1:] = uini(x)
    u = upwind(u, nt, dtoverdx)
    
    marker=dict(symbol='x')
    fig.add_trace(go.Scatter(x=x, y=u[1:], name='num sol.', visible=False, mode='markers', marker=marker))

fig.data[2].visible = True

# Create and add slider
steps = []
for i, nx_i in enumerate(nx):
    args = [{"visible": [(el==0 or el==1 or el==i+2) for el in range(len(fig.data))]}]
    step = dict(method="update", label = f"{nx_i}", args=args)
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'nx = '}, steps=steps)]

fig.update_layout(sliders=sliders, title=f"Schéma upwind : cfl={cfl}")
fig.show()